### There is a problem with Q3 - the price already factors in quantity.  Need to fix.

brendanbailey [11:03 AM] Tue Sep 5, 2017

Hello Joanne

I have your unit 2 feedback.

8/8!

+s: Great job in solving these problems!
Great analysis on the bonus seeing which items are the most popular. I gave an extra point for the bonus to offset the error on question 3.

-s: So quantity is already taken into account in the item price for question 3. That's why for a Chicken Bowl price is around `$`8 for 1 bowl and about `$`16 for 2 bowls. Despite this you multiplied the item price by quantity thus inflating the item price. The correct answer is `$`18.81.

Amazing :rainbow:

<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [30]:
import csv

master_list = []   # list storage for tsv file

with open('data/chipotle.tsv') as csvfile:
    file_nested_list = csv.reader(csvfile, delimiter='\t')
    for rec in file_nested_list:
        master_list.append(rec)

In [228]:
# first 5 items in the list of lists
master_list[:5]

[['order_id', 'quantity', 'item_name', 'choice_description', 'item_price'],
 ['1', '1', 'Chips and Fresh Tomato Salsa', 'NULL', '$2.39 '],
 ['1', '1', 'Izze', '[Clementine]', '$3.39 '],
 ['1', '1', 'Nantucket Nectar', '[Apple]', '$3.39 '],
 ['1', '1', 'Chips and Tomatillo-Green Chili Salsa', 'NULL', '$2.39 ']]

### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [31]:
# header is the first record in the dataset
header = master_list[0]
header

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

In [32]:
# data includes all records after the header
data = master_list[1:]
data[:5]   # shows first 5 records in the data dataset

[['1', '1', 'Chips and Fresh Tomato Salsa', 'NULL', '$2.39 '],
 ['1', '1', 'Izze', '[Clementine]', '$3.39 '],
 ['1', '1', 'Nantucket Nectar', '[Apple]', '$3.39 '],
 ['1', '1', 'Chips and Tomatillo-Green Chili Salsa', 'NULL', '$2.39 '],
 ['2',
  '2',
  'Chicken Bowl',
  '[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]',
  '$16.98 ']]

---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [69]:
# assumes that the order_id is sorted in ascending order and every order_id in the sequence exists

curr_order_id = 0          # order_id pointer tracks order_id (many recs per order_id)
grand_total = 0            # total of all orders
num_orders = 0             # number of orders to average over
avg_order_amt = 0          # final answer: the average price of an order

for rec in data:
    
    curr_order_amt = 0     # cumulative over all recs in an order
    curr_rec_qty = 0       # item quantity to be multiplied by item price
    curr_rec_price = 0     # item price
    curr_rec_total = 0     # record total is qty * price for item
    
    rec_order_id = int(rec[int(header.index('order_id'))])
    
    # when current order_id pointer does not match reference pointer, 
    #  we have moved onto a record that has a different order_id
    
    # initializes order per order_id:
    if rec_order_id != curr_order_id:
        num_orders += 1
        curr_order_id += 1

    # calculates and updates amounts per order:
    curr_rec_qty = float(rec[int(header.index('quantity'))])
    curr_rec_price = float(rec[int(header.index('item_price'))][1:])        
    curr_rec_total = curr_rec_qty * curr_rec_price
    grand_total += curr_rec_total

avg_order_amt = round(grand_total / num_orders, 2)
print 'Average Price of an Order: $' + str(grand_total) + ' / ' + str(num_orders) + ' = $' + str(avg_order_amt)

Average Price of an Order: $39237.02 / 1834 = $21.39


### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [70]:
all_sodas = []     # holds all sodas and soft drinks from dataset (includes duplicates)

# generates list of all sodas and soft drinks from each record in the dataset (includes duplicates)
for order in data:
    if (order[int(header.index('item_name'))] == 'Canned Soda') or (order[int(header.index('item_name'))] == 'Canned Soft Drink'):
        all_sodas.append(order[int(header.index('choice_description'))])
        
# creates unique set of all sodas (removes duplicates)
sodas = set(all_sodas)

# shows results of number of sodas and soft drinks sold at Chipotle
print 'Chipotle sells', len(sodas), 'kinds of sodas and soft drinks:'
for soda in sodas:
    print ' ',soda[1:-1]

Chipotle sells 9 kinds of sodas and soft drinks:
  Lemonade
  Dr. Pepper
  Diet Coke
  Nestea
  Mountain Dew
  Diet Dr. Pepper
  Coke
  Coca Cola
  Sprite


---

## Advanced Level


### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


In [352]:
# disregards 'quantity'
num_burritos = 0
total_toppings = 0
avg_toppings_per_burrito = 0

for item in data:
    
    # flattens choice_description and counts toppings for burritos
    if 'Burrito' in item[int(header.index('item_name'))]:
        # toppings list is actually a string; removes '[' and ']' in case actual items matter, later
        toppings = item[int(header.index('choice_description'))].replace('[','').replace(']','').split(',')
        num_toppings = len(toppings)
        total_toppings += num_toppings
        num_burritos += 1

avg_toppings_per_burrito = total_toppings / num_burritos
print 'Average toppings per burrito: ', total_toppings, 'toppings /', num_burritos, 'burritos = ', avg_toppings_per_burrito

Average toppings per burrito:  6323 toppings / 1172 burritos =  5


### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

In [68]:
from collections import defaultdict

chips_dict = defaultdict(int)

for item in data:
    if 'Chips' in item[int(header.index('item_name'))]:
        curr_chip_qty = int(item[int(header.index('quantity'))])
        # consolidates same chip types that contain '-'
        chips_dict[item[int(header.index('item_name'))].replace('-',' ')] += curr_chip_qty

print chips_dict, '\n'
for chip, amt  in chips_dict.items():
    print amt, chip

defaultdict(<type 'int'>, {'Chips and Mild Fresh Tomato Salsa': 1, 'Chips and Guacamole': 506, 'Chips and Fresh Tomato Salsa': 130, 'Chips and Tomatillo Red Chili Salsa': 75, 'Side of Chips': 110, 'Chips and Roasted Chili Corn Salsa': 41, 'Chips': 230, 'Chips and Tomatillo Green Chili Salsa': 78}) 

1 Chips and Mild Fresh Tomato Salsa
506 Chips and Guacamole
130 Chips and Fresh Tomato Salsa
75 Chips and Tomatillo Red Chili Salsa
110 Side of Chips
41 Chips and Roasted Chili Corn Salsa
230 Chips
78 Chips and Tomatillo Green Chili Salsa


---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!


Create a list of single ingredient toppings (burritos, bowls, tacos, salads) by order of popularity, with consideration to order quantity.  Remove multi-item toppings from the choice_description list or sublist.

In [176]:
from collections import defaultdict

single_ingredients = defaultdict(int)

for rec in data:

    # examines menu items that come with toppings (burritos, bowls, tacos, and salads) and their quantity, 
    #  as number of servings.  
    if rec[int(header.index('item_name'))].split(' ')[-1] in ['Burrito', 'Bowl', 'Tacos', 'Salad']:
        item_servings = int(rec[int(header.index('quantity'))])
        # sub list is actually a string, so removes square brackets and creates a flat list of ingredients
        item_choices = rec[int(header.index('choice_description'))].replace('[','').replace(']','').strip().split(', ')

        for item in item_choices:
            # removes multi-ingredient toppings
            if item.find('Salsa') == -1 \
                and item.find('Fajita') == -1 \
                and item.find('Carnitas') == -1 \
                and item.find('Chili') == -1 \
                and item.find('Barbacoa') == -1 \
                and item.find('-') == -1:
                # updates single ingredient's serving count / quantity
                single_ingredients[item] += item_servings

print single_ingredients, '\n'

# order by popularity - sorts single_ingredients dictionary by value
for servings in sorted(single_ingredients, key=single_ingredients.get, reverse=True):
      print str(single_ingredients[servings]).rjust(5), servings

defaultdict(<type 'int'>, {'Pinto Beans': 608, 'Cheese': 2389, 'Sour Cream': 1785, 'Guacamole': 1077, 'Brown Rice': 2, 'Fresh Tomato (Mild)': 33, 'Vegetarian Black Beans': 3, 'Lettuce': 1804, 'Black Beans': 1412, 'Rice': 2497, 'White Rice': 5}) 

 2497 Rice
 2389 Cheese
 1804 Lettuce
 1785 Sour Cream
 1412 Black Beans
 1077 Guacamole
  608 Pinto Beans
   33 Fresh Tomato (Mild)
    5 White Rice
    3 Vegetarian Black Beans
    2 Brown Rice
